In [14]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, default_collate
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, root_dir, split='Training', transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.labels = {'정상': 0, '균핵병': 1, '노균병': 2}
        self.data = []
        self._load_dataset()

    def _load_dataset(self):
        for label_name, label in self.labels.items():
            path = os.path.join(self.root_dir, '원천데이터', '05.상추', f'05.상추_{label}.{label_name}')
            images = os.listdir(path)
            for image in images:
                self.data.append((os.path.join(path, image), label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [15]:
# DataLoader에서 None 타입 데이터를 걸러내기 위한 함수
def my_collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    return default_collate(batch)

# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 및 DataLoader 인스턴스 생성
root_dir = '/Users/leeyeonju/Desktop/smart-plant-ai/smart_plant_ai/realdata'
train_dataset = CustomDataset(root_dir=root_dir, split='Training', transform=transform)
val_dataset = CustomDataset(root_dir=root_dir, split='Validation', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=my_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=my_collate_fn)


In [16]:
# 모델, 손실 함수, 최적화 알고리즘 설정
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)  # 전이학습 - 가중치 그대로 사용
num_ftrs = model.fc.in_features  # 모델의 마지막 fully connected층의 입력 특성 수 가져옴
model.fc = nn.Linear(num_ftrs, 3)  # 원래 모델의 마지막 fc레이어를 새로운 레이어로 대체 - num_ftrs개의 입력 특성을 받아 3개의 출력을 가정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류에 적합한 손실함수
optimizer = optim.Adam(model.parameters(), lr=1e-4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 훈련 및 검증 루프
num_epochs = 10
best_cost = 1e+10
for epoch in range(num_epochs):
    print("================")
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # 모델을 학습 모드로 설정
        else:
            model.eval()   # 모델을 평가 모드로 설정

        running_loss = 0.0
        corrects = 0
        sample_size = 0

        # 데이터 로더 설정
        dataloader = train_loader if phase == 'train' else val_loader

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # forward
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * images.size(0)
            corrects += torch.sum(preds == labels.data)
            sample_size += images.size(0)

        epoch_loss = running_loss / sample_size
        epoch_acc = corrects.double() / sample_size

        print(f"Epoch: {epoch+1}/{num_epochs}, Phase: {phase}, Loss: {epoch_loss}, ACC: {epoch_acc}")

        # 모델 저장
        if phase == 'val' and epoch_loss < best_cost:
            print("Model saved")
            best_cost = epoch_loss
            torch.save(model.state_dict(), 'model.pt')

Epoch: 1/10, Phase: train, Loss: 0.7718769501756739, ACC: 0.7051851851851851
Epoch: 1/10, Phase: val, Loss: 0.5412274956703186, ACC: 0.8333333333333334
Model saved
Epoch: 2/10, Phase: train, Loss: 0.2516471983326806, ACC: 0.9362962962962963
Epoch: 2/10, Phase: val, Loss: 0.3869178838200039, ACC: 0.8555555555555555
Model saved
Epoch: 3/10, Phase: train, Loss: 0.07735049975139123, ACC: 0.9896296296296296
Epoch: 3/10, Phase: val, Loss: 0.4657071338759528, ACC: 0.8444444444444444
Epoch: 4/10, Phase: train, Loss: 0.0499773649043507, ACC: 0.9940740740740741
Epoch: 4/10, Phase: val, Loss: 0.5377604391839769, ACC: 0.8444444444444444
Epoch: 5/10, Phase: train, Loss: 0.038128319460277754, ACC: 0.9881481481481481
Epoch: 5/10, Phase: val, Loss: 0.3816556341118283, ACC: 0.8333333333333334
Model saved
Epoch: 6/10, Phase: train, Loss: 0.04473652953350985, ACC: 0.9881481481481481
Epoch: 6/10, Phase: val, Loss: 0.4366233189900716, ACC: 0.8444444444444444
Epoch: 7/10, Phase: train, Loss: 0.0420444883816

In [24]:
import torch
from torchvision import transforms
from PIL import Image

# 이미지를 불러오고 전처리하기 위한 transform 설정
transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image(image_path):
    # 이미지 불러오기 및 변환
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원 추가

    # 모델로 예측
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        outputs = model(image.to(device))
        _, preds = torch.max(outputs, 1)

    # 예측된 라벨 반환
    return preds.item()

# 모델 가중치 불러오기 (앞서 저장한 'model.pt')
model.load_state_dict(torch.load('model.pt', map_location=device))

# 예측할 이미지의 경로 설정
image_path = '/Users/leeyeonju/Desktop/smart-plant-ai/smart_plant_ai/KakaoTalk_Photo_2024-05-14-02-06-39.jpeg'

# 이미지에 대한 예측 라벨 출력
predicted_label = predict_image(image_path)
print(f'Predicted label: {predicted_label}')


Predicted label: 0


In [26]:
# 테스트 데이터셋과 데이터로더 설정
test_dataset = CustomDataset(root_dir=root_dir, split='Test', transform=transform)  # 테스트 데이터셋 추가
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=my_collate_fn)  # 테스트 DataLoader 추가

# 모델 로드
model.load_state_dict(torch.load('model.pt'))
model.eval()  # 모델을 평가 모드로 설정
model.to(device)

# 테스트 성능 평가
running_loss = 0.0
corrects = 0
sample_size = 0

for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    batch_loss = loss.item() * images.size(0)
    batch_corrects = torch.sum(preds == labels.data)
    
    running_loss += batch_loss
    corrects += batch_corrects
    sample_size += images.size(0)
    
    # 현재 배치에 대한 손실과 정확도 출력
    print(f"Batch Loss: {batch_loss / images.size(0)}, Batch Accuracy: {batch_corrects.double() / images.size(0)}")

# 전체 테스트 데이터셋에 대한 손실과 정확도 계산
test_loss = running_loss / sample_size
test_acc = corrects.double() / sample_size

print(f"Total Test Loss: {test_loss}, Total Test Accuracy: {test_acc}")


Batch Loss: 0.35215914249420166, Batch Accuracy: 0.875
Batch Loss: 0.5211045742034912, Batch Accuracy: 0.75
Batch Loss: 0.12497228384017944, Batch Accuracy: 1.0
Batch Loss: 0.2852169871330261, Batch Accuracy: 0.8125
Batch Loss: 1.9388319253921509, Batch Accuracy: 0.4375
Batch Loss: 1.3500956296920776, Batch Accuracy: 0.4
Total Test Loss: 0.7228612767325507, Total Test Accuracy: 0.7333333333333333


드롬아웃 정규화 과정 추가

In [1]:
import os
from torchvision import transforms, models
import torch
from torch.utils.data import Dataset, DataLoader, default_collate
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import torch.nn as nn
import torch.optim as optim
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, root_dir, split='Training', transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.labels = {'정상': 0, '균핵병': 1, '노균병': 2}
        self.data = []
        self._load_dataset()

    def _load_dataset(self):
        for label_name, label in self.labels.items():
            path = os.path.join(self.root_dir, '원천데이터', '05.상추', f'05.상추_{label}.{label_name}')
            images = os.listdir(path)
            for image in images:
                self.data.append((os.path.join(path, image), label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
def my_collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    return default_collate(batch)

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

root_dir = '/Users/leeyeonju/Desktop/smart-plant-ai/smart_plant_ai/modeldata'
train_dataset = CustomDataset(root_dir=root_dir, split='Training', transform=transform)
val_dataset = CustomDataset(root_dir=root_dir, split='Validation', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=my_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=my_collate_fn)


In [4]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# 모델 수정: 마지막 fc 레이어에 드롭아웃 추가
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # 50% 드롭아웃 추가
    nn.Linear(num_ftrs, 3)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
best_cost = 1e+10
for epoch in range(num_epochs):
    print("================")
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        corrects = 0
        sample_size = 0

        dataloader = train_loader if phase == 'train' else val_loader

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * images.size(0)
            corrects += torch.sum(preds == labels.data)
            sample_size += images.size(0)

        epoch_loss = running_loss / sample_size
        epoch_acc = corrects.double() / sample_size

        print(f"Epoch: {epoch+1}/{num_epochs}, Phase: {phase}, Loss: {epoch_loss}, ACC: {epoch_acc}")

        if phase == 'val' and epoch_loss < best_cost:
            print("Model saved")
            best_cost = epoch_loss
            torch.save(model.state_dict(), 'model.pt')

Epoch: 1/10, Phase: train, Loss: 0.42168913591206464, ACC: 0.847167325428195
Epoch: 1/10, Phase: val, Loss: 0.1720637880645676, ACC: 0.9512195121951219
Model saved
Epoch: 2/10, Phase: train, Loss: 0.12582179702995086, ACC: 0.9581686429512516
Epoch: 2/10, Phase: val, Loss: 0.14616462485678908, ACC: 0.9490022172949002
Model saved
Epoch: 3/10, Phase: train, Loss: 0.07149691717534747, ACC: 0.9782608695652174
Epoch: 3/10, Phase: val, Loss: 0.10809867431362162, ACC: 0.9578713968957872
Model saved
Epoch: 4/10, Phase: train, Loss: 0.04974500677880876, ACC: 0.9855072463768116
Epoch: 4/10, Phase: val, Loss: 0.12119451209390639, ACC: 0.9578713968957872
Epoch: 5/10, Phase: train, Loss: 0.039559953635600206, ACC: 0.991106719367589
Epoch: 5/10, Phase: val, Loss: 0.15483482419915845, ACC: 0.9445676274944568
Epoch: 6/10, Phase: train, Loss: 0.021660008480924527, ACC: 0.994729907773386
Epoch: 6/10, Phase: val, Loss: 0.16063757690236058, ACC: 0.9445676274944568
Epoch: 7/10, Phase: train, Loss: 0.0224740

In [41]:
import torch
from torchvision import transforms
from PIL import Image

# 이미지를 불러오고 전처리하기 위한 transform 설정
transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image(image_path):
    # 이미지 불러오기 및 변환
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원 추가

    # 모델로 예측
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        outputs = model(image.to(device))
        _, preds = torch.max(outputs, 1)

    # 예측된 라벨 반환
    return preds.item()

# 모델 가중치 불러오기 (앞서 저장한 'model.pt')
model.load_state_dict(torch.load('model.pt', map_location=device))

# 예측할 이미지의 경로 설정
image_path = '/Users/leeyeonju/Desktop/smart-plant-ai/이연주 서명.png'

# 이미지에 대한 예측 라벨 출력
predicted_label = predict_image(image_path)
print(f'Predicted label: {predicted_label}')


Predicted label: 2


In [44]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# 모델 구조를 재정의
def initialize_model():
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),  # 50% 드롭아웃 추가
        nn.Linear(num_ftrs, 3)
    )
    return model

# 이미지를 불러오고 전처리하기 위한 transform 설정
transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 모델 로드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = initialize_model()  # 모델 구조 초기화
model.load_state_dict(torch.load('model.pt', map_location=device))  # 학습된 가중치 로드
model.to(device)
model.eval()

def predict_image(image_path):
    # 이미지 불러오기 및 변환
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # 배치 차원 추가

    # 모델로 예측
    with torch.no_grad():
        outputs = model(image.to(device))
        _, preds = torch.max(outputs, 1)

    # 예측된 라벨 반환
    return preds.item()

# 예시 사용
# 예측 결과 = predict_image('path_to_your_image.jpg')
# print("Predicted label:", 예측 결과)


res = predict_image('/Users/leeyeonju/Desktop/smart-plant-ai/smart_plant_ai/realdata/Training/원천데이터/05.상추/05.상추_2.노균병/V006_77_1_10_05_03_12_1_0149e_20201209_14.jpeg')
print(res)

2


In [5]:
# 테스트 데이터셋과 데이터로더 설정
test_dataset = CustomDataset(root_dir=root_dir, split='Test', transform=transform)  # 테스트 데이터셋 추가
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=my_collate_fn)  # 테스트 DataLoader 추가

# 모델 로드
model.load_state_dict(torch.load('model.pt'))
model.eval()  # 모델을 평가 모드로 설정
model.to(device)

# 테스트 성능 평가
running_loss = 0.0
corrects = 0
sample_size = 0

for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    batch_loss = loss.item() * images.size(0)
    batch_corrects = torch.sum(preds == labels.data)
    
    running_loss += batch_loss
    corrects += batch_corrects
    sample_size += images.size(0)
    
    # 현재 배치에 대한 손실과 정확도 출력
    print(f"Batch Loss: {batch_loss / images.size(0)}, Batch Accuracy: {batch_corrects.double() / images.size(0)}")

# 전체 테스트 데이터셋에 대한 손실과 정확도 계산
test_loss = running_loss / sample_size
test_acc = corrects.double() / sample_size

print(f"Total Test Loss: {test_loss}, Total Test Accuracy: {test_acc}")

Batch Loss: 0.3694264888763428, Batch Accuracy: 0.875
Batch Loss: 0.1767321527004242, Batch Accuracy: 0.9375
Batch Loss: 0.1822088062763214, Batch Accuracy: 0.9375
Batch Loss: 0.12000128626823425, Batch Accuracy: 1.0
Batch Loss: 0.07223469018936157, Batch Accuracy: 1.0
Batch Loss: 0.04182419180870056, Batch Accuracy: 1.0
Batch Loss: 0.05307520553469658, Batch Accuracy: 1.0
Batch Loss: 0.15647490322589874, Batch Accuracy: 0.9375
Batch Loss: 0.12362276017665863, Batch Accuracy: 0.9375
Batch Loss: 0.17788153886795044, Batch Accuracy: 0.875
Batch Loss: 0.04220421984791756, Batch Accuracy: 1.0
Batch Loss: 0.1340942531824112, Batch Accuracy: 0.875
Batch Loss: 0.1630210280418396, Batch Accuracy: 0.875
Batch Loss: 0.05923359841108322, Batch Accuracy: 1.0
Batch Loss: 0.21102644503116608, Batch Accuracy: 0.9375
Batch Loss: 0.04461194574832916, Batch Accuracy: 1.0
Batch Loss: 0.12695112824440002, Batch Accuracy: 0.9375
Batch Loss: 0.08250182867050171, Batch Accuracy: 0.9375
Batch Loss: 0.10959117